In [1]:
from RAG.qdrant_database import QdrantDatabase

In [2]:
from qdrant_client import models

In [ ]:
import os
api = os.environ.get('DB_API')

from qdrant_client import QdrantClient

qd = QdrantDatabase(
    cluster_uri="Cluster uri here!",
    api=api
)

c:\Users\91790\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
from sentence_transformers import SentenceTransformer

encoder = SentenceTransformer('all-MiniLM-L6-v2')

In [5]:
# get collection 
qd.client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='furr_bot')])

In [3]:
lst = ['a', 'b', 'c', 'd', 'e']
print(lst[-2])

d


**Custom Re-Ranking**

In [50]:
from openai import OpenAI 
# check time for code run
import time

def chat(prompt):
    client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))
    completion = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {
                        "role": "system",
                        "content": "You are a member of a chatbot system, your responsibility is to analyze the previous conversation and current user query, and determine whether the user is asking for a new information or wants to continue with previous conversation"},
                    {
                        "role": "user",
                        "content": prompt
                    }
                ]
            )
    answer = completion.choices[0].message.content
    return answer


def do_retrieval(conversation, query):
    prompt = f"""  
                Determine whether user is asking for new information or wants to continue with previous conversation.
                Conversation: {conversation}
                Query: {query}
                Your answer should be either "True" for new information or "False" for previous conversation.
            """
    answer = chat(prompt)
    return answer

In [53]:
con = "How should i feed my hamster?"
query = "how to train my him?"

do_retrieval(con, query)

'True'

In [39]:
query = "so how should i treat him?"
res = qd.client.query_points(
    collection_name ="furr_bot",
    query = encoder.encode(query).tolist(),
    limit=3,
    query_filter = models.Filter(
        must=[
            models.FieldCondition(
                key="word_count",
                range=models.Range(
                    gte=40, 
                    lte=300
                )
            )
        ]
    )
)

In [40]:
ans = res.points[:2]

In [41]:
ans[0].payload

{'text': "treatment the anchor worm can be easily removed by carefully pulling it out from the fish's skin. the infected area is then treated with a topical antibiotic ointment. afterwards, the pond or aquarium should be sanitized and disinfected, in an effort to remove any adult parasites, larvae or eggs. /24, ",
 'topic': 'Treatment',
 'blog_name': 'Anchor Worm Symptoms - Fish',
 'pet': 'Fish',
 'word_count': 50,
 'chunk_size': 58}

In [37]:
paras = []
for a in ans:
    d = a.payload
    d['score'] = a.score
    paras.append(
        d
    )

In [38]:
paras

[{'text': "living and management first and foremost, your hamster needs plenty of rest in a calm, clean, and sanitary environment. consult your veterinarian to formulate a diet that meets the hamster's special needs and follow any other instructions your veterinarian may have given. /24, ",
  'topic': 'Living and Management',
  'blog_name': 'Inflammation of the Kidneys in Hamsters',
  'pet': 'Hamsters',
  'word_count': 43,
  'chunk_size': 50,
  'score': 0.72854924},
 {'text': "living and management while recovering from the toxicity, the hamster should be monitored for symptoms indicating an allergic relapse. in addition, consult your veterinarian about the hamster's dietary requirements during the recovery period. if your pet refuses to eat, /24, force-feeding may be necessary.",
  'topic': 'Living and Management',
  'blog_name': 'Antibiotics-Induced Enteritis in Hamsters',
  'pet': 'Hamsters',
  'word_count': 44,
  'chunk_size': 53,
  'score': 0.7004651}]